# RAG Manual de Tránsito

In [23]:
import os
from dotenv import load_dotenv
from pathlib import Path

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_core.documents import Document

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from langchain.load import dumps, loads

from operator import itemgetter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

import re

In [24]:
# Define the path to the .env file (one directory above)
dotenv_path = '.env'

# Load the .env file
load_dotenv(dotenv_path)

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')

In [25]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langsmith_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [12]:
# #### INDEXING ####

# # Load local text file
# file_path = "ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt"
# loader = TextLoader(file_path)
# docs = loader.load()

# # Split the text
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=300,
#     chunk_overlap=50
# )

# # Make splits
# splits = text_splitter.split_documents(docs)

# # Index
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=OpenAIEmbeddings()
# )

# retriever = vectorstore.as_retriever()

In [13]:
# Load local text file
file_path = "ley-769-de-2002-codigo-nacional-de-transito_3704_0_processed.txt"
loader = TextLoader(file_path)
docs = loader.load()

def clean_section_text(text):
    """
    Remove the article header up to the next uppercase letter.
    """
    # Pattern to match from article header to next uppercase letter
    header_pattern = r'^(?:ARTÍCULO|ARTICULO|Artículo|Articulo)\s+\d+.*?(?=[A-ZÁÉÍÓÚÑ])'
    cleaned_text = re.sub(header_pattern, '', text, flags=re.DOTALL)
    return cleaned_text.strip()

def extract_article_sections(text):
    """
    Extract sections of text between article markers and assign article numbers.
    Only matches when starting with capital A ('ARTÍCULO' or 'Artículo').
    Returns a list of (text, article_number) tuples with cleaned section text.
    """
    # Pattern that only matches when starting with capital A
    article_pattern = r'(?:ARTÍCULO|ARTICULO|Artículo|Articulo)\s+(\d+)'
    
    # Find all article markers with their positions
    article_matches = list(re.finditer(article_pattern, text))
    
    article_sections = []
    
    # Process each article section
    for i in range(len(article_matches)):
        start_pos = article_matches[i].start()
        article_num = article_matches[i].group(1)

        # Get end position (either next article or end of text)
        if i < len(article_matches) - 1:
            end_pos = article_matches[i + 1].start()
        else:
            end_pos = len(text)
        
        section_text = text[start_pos:end_pos]
        # Clean the section text by removing the header
        cleaned_section = clean_section_text(section_text)
        article_sections.append((cleaned_section, article_num))
    
    return article_sections

# Process the original text to get article sections
original_text = docs[0].page_content
article_sections = extract_article_sections(original_text)

# Modified text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

# Create splits with article metadata
splits = []
for section_text, article_num in article_sections:
    section_splits = text_splitter.create_documents(
        texts=[section_text],
        metadatas=[{
            "source": file_path,
            "source_article": article_num
        }]
    )
    splits.extend(section_splits)

# Index with the enhanced metadata
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

retriever = vector_store.as_retriever()

### Define state

In [15]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [20]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs with article numbers """
    # Flatten list of lists, and convert each Document to string
    # We'll add the article number to the string representation to make it part of uniqueness check
    flattened_docs = [dumps(doc.page_content + doc.metadata.get('source_article', '')) 
                    for sublist in documents 
                    for doc in sublist]
    print("Total number of docs retrieved", len(flattened_docs))
    unique_docs = list(set(flattened_docs))
    print("Total number of unique docs retrieved", len(unique_docs))

    # When loading back, we need to split the content and metadata
    loaded_unique_docs = []
    for doc_str in unique_docs:
        doc = loads(doc_str)
        # Get original document from documents list to preserve metadata
        original_doc = next((d for sublist in documents 
                        for d in sublist 
                        if d.page_content + d.metadata.get('source_article', '') == doc), None)
        if original_doc:
            loaded_unique_docs.append(original_doc)

    for doc in loaded_unique_docs:
        print("=======================")
        print(f"Article {doc.metadata.get('source_article', 'N/A')}:")
        print(doc.page_content)

    return loaded_unique_docs


# Define application steps
def retrieve(state: State):
    # Multi Query: Different Perspectives
    template = """Eres una IA asistente modelo de lenguaje. Tu tarea es generar cinco
    diferentes versiones de la pregunta dada por el usuario para extraer los documentos
    relevantes de una base de datos de vectores. Al generar múltiples perspectivas
    de la pregunta del usuario, tu objetivo es ayudar al usuario a superar algunas
    de las limitaciones de la búsqueda de similaridad basada en distancia. Escribe
    estas preguntas alternativas separadas por caracteres de nueva línea, sin enumerar
    ni listar. Pregunta original: {question}"""

    prompt_perspectives = ChatPromptTemplate.from_template(template)

    generate_queries = (
        prompt_perspectives
        | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    )

    # retriever.map is the one that brings the list of documents most similar
    # to each of the queries generated by generate_queries

    retrieval_chain = generate_queries | retriever.map() | get_unique_union
    docs = retrieval_chain.invoke({"question": state["question"]})

    return {"context": docs}

# Modified chain to format context with article numbers
def format_context_with_articles(docs):
    formatted_contexts = []
    for doc in docs:
        article_num = doc.metadata.get('source_article', 'N/A')
        formatted_contexts.append(f"[Artículo {article_num}] {doc.page_content}")

    print("++++++++++++++++++")
    print("\n\n".join(formatted_contexts))
    return "\n\n".join(formatted_contexts)


def generate(state: State):
    # Modified template to include article references
    template = """Responde la siguiente pregunta basado en este contexto, sin decir
    que te basaste en el contexto, y mencionando solo lo que te parece relacionado.
    Primero menciona la respuesta rápida (si la hay),
    y después da un poco más de detalles, mencionando el artículo en que te basaste
    en cada parte de la respuesta al principio de dicha parte, diciendo "Basado en
    el artículo X,..."

    Contexto con artículos:
    {context}

    Pregunta: {question}
    """
    
    context = format_context_with_articles(state["context"])

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI(model_name="gpt-4", temperature=0)

    generation_rag_chain = (
        prompt
        | llm
        # | StrOutputParser()
    )

    response = generation_rag_chain.invoke({
        "context": context,
        "question": state["question"]
    })

    return {"answer": response.content}


    # docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    # messages = prompt.invoke({"question": state["question"], "context": docs_content})
    # response = llm.invoke(messages)
    # return {"answer": response.content}

### Build Graph

In [21]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [22]:
response = graph.invoke({"question": "una moto puede ir por la linea amarilla entre carros"})
print(response["answer"])

Total number of docs retrieved 20
Total number of unique docs retrieved 6
Article 96:
NORMAS ESPECÍFICAS PARA MOTOCICLETAS,
MOTOCICLOS Y MOTOTRICICLOS. Las motocicletas se sujetarán a las siguientes normas específicas: 
1. Podrán llevar un acompañante en su vehículo, el cual también deberá utilizar casco y elementos de seguridad.
2. Deberán usar de acuerdo con lo estipulado para vehículos automotores, las luces direccionales.
3. Cuando transiten por las vías de uso público deberán hacerlo con las luces delanteras y traseras encendidas.
4. El conductor deberá portar siempre chaleco reflectivo identificado con el número de la placa del vehículo en que se transite
CAPITULO VI.
TRÁNSITO DE OTROS VEHÍCULOS Y DE ANIMALES.
Article 118:
Amarilla: Indica atención para un cambio de luces o señales y para que el cruce sea desalojado por los vehículos que se encuentran en él o se abstengan de ingresar en el cruce aun disponiendo de espacio para hacerlo. No debe iniciarse la marcha en luz amarilla,